In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = bassamsabber
%env KAGGLE_KEY = 717678f8ebfb53d84e9c47444c275871
!kaggle competitions download -c computationalintelligencesc2020
!unzip DataSet
!unzip computationalintelligencesc2020

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import os, cv2
import tensorflow as tf
import keras
from tqdm import tqdm

In [ ]:
code = {'buildings':0 ,'forest':1,'glacier':2,'mountain':3,'sea':4,'street':5}

In [ ]:
IMG_SIZE = 100
DATA_DIR = r'/content/Scenes training set/Scenes training set'
size = []
categories = ['buildings','forest','glacier','mountain','sea','street']
training_data = []

for category in categories:
  path = os.path.join(DATA_DIR, category)
  class_label = categories.index(category)

  for img in tqdm(os.listdir(path)):
    try:
      img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
      img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
      size.append(img_arr.shape)
      training_data.append([img_arr, class_label])
    except Exception as e:
      pass

val_cnts = pd.Series(size).value_counts()
print(val_cnts)

In [ ]:
import random
random.shuffle(training_data)
#print(training_data[:10])
X = []
y = []

for features, label in training_data:
  X.append(features)
  y.append(label)

In [ ]:
X = np.array(X).reshape(-1,100,100,1)
y = np.array(y)
#X =np.expand_dims(X, axis=1)
print(f'X_train shape  is {X.shape}')
print(f'y_train shape  is {y.shape}')

In [ ]:
model = keras.models.Sequential([
        keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,1)),
        keras.layers.Conv2D(150,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Conv2D(120,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(50,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(120,activation='relu') ,    
        keras.layers.Dense(100,activation='relu') ,    
        keras.layers.Dense(50,activation='relu') ,        
        keras.layers.Dropout(rate=0.5) ,            
        keras.layers.Dense(6,activation='softmax') ,    
        ])


In [ ]:
model.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print('Model Details are : ')
print(model.summary())

In [ ]:
epochs = 20
ThisModel = model.fit(X, y, epochs=epochs,batch_size=64,verbose=1)

In [ ]:
DATA_DIR_TEST = r'/content/Scenes testing test/Scenes testing test'
images = []
testing_data = []
for img in tqdm(os.listdir(DATA_DIR_TEST)):
  try:
    images.append(img)
    img_arr = cv2.imread(os.path.join(DATA_DIR_TEST, img), cv2.IMREAD_GRAYSCALE)
    img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
    testing_data.append(img_arr)
  except Exception as e:
    pass
len(testing_data)

In [ ]:
testing_data = np.array(testing_data).reshape(-1,100,100,1)

In [ ]:
outputs = model.predict(testing_data)
final_outputs = np.argmax(outputs, axis=1)
submit = pd.read_csv('/content/submit.csv')
submit['Image'] = images
submit['Label'] = final_outputs
submit.to_csv("submit.csv")
print("Done!")